# SETUP 

In [1]:
import sys
import os
from pathlib import Path

modulepath = Path.cwd().parent / "utils"
modulepath = str(modulepath)
projectpath = Path.cwd().parent
projectpath = str(projectpath)
if modulepath not in sys.path:
    sys.path.insert(0,str(modulepath))
if projectpath not in sys.path:
    sys.path.insert(0,str(projectpath))  
print(sys.path)

['c:\\Users\\oriol\\GitHub_Repos\\CNN-classification', 'c:\\Users\\oriol\\GitHub_Repos\\CNN-classification\\utils', 'c:\\Users\\oriol\\anaconda3\\envs\\pytorch\\python312.zip', 'c:\\Users\\oriol\\anaconda3\\envs\\pytorch\\DLLs', 'c:\\Users\\oriol\\anaconda3\\envs\\pytorch\\Lib', 'c:\\Users\\oriol\\anaconda3\\envs\\pytorch', '', 'c:\\Users\\oriol\\anaconda3\\envs\\pytorch\\Lib\\site-packages', 'c:\\Users\\oriol\\anaconda3\\envs\\pytorch\\Lib\\site-packages\\win32', 'c:\\Users\\oriol\\anaconda3\\envs\\pytorch\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\oriol\\anaconda3\\envs\\pytorch\\Lib\\site-packages\\Pythonwin']


# TRAIN

In [2]:
import torch
from torch import nn

from utils.engine import train
from utils.model_architectures import TinyVGG_1
from utils.data_loaders import create_train_test_dataloaders
from utils.helpers import create_writer
from utils.helpers import create_dataframe
from utils.helpers import hyperparameter_combinations
from utils.helpers import save_dataframe
from utils.helpers import create_and_save_experiment_metadata

device = "cuda" if torch.cuda.is_available() else "cpu"
#SEED = 0
#torch.manual_seed(SEED)
# There are other sources of randomness, so manual_seed does not enable 
# reproducibility

#next(model.parameters()).is_cuda #check if model on cuda
#from torchinfo import summary
#summary(tvgg,input_size=[32,1,28,28])

# A Hyperparameter named Iter controls the number of iterations for a given
# set of hyperparameters
HYPERPARAMETERS = {'Hidden_Channels': [20,30,40],
                   'Epochs': [3,4],
                   'lr': [0.1,0.01,0.001],
                   'Iter': [2],
                   'Size': [0.05],
                   }

experiment_name = "Big_sweep"
hp_combinations = hyperparameter_combinations(HYPERPARAMETERS)
hp_keys = HYPERPARAMETERS.keys()

for combination in hp_combinations:
    HIDDEN_CHANNELS = combination["Hidden_Channels"]
    NUM_EPOCHS = combination["Epochs"]
    LR = combination["lr"]
        # HIDDEN_CHANNELS, NUM_EPOCHS, LR have to be specified, cannot be none
    ITER = combination["Iter"] if "Iter" in hp_keys else None 
        # ITER Defaults to 1 when None
    SIZE = combination["Size"] if "Size" in hp_keys else None 
        # SIZE Defaults to 1.0 when None

    train_dataloader, test_dataloader = create_train_test_dataloaders(size=SIZE) #type: ignore

    # create multiple models and optimizers, one pair for every iteration
    model_0 = []
    optimizer_0 = []
    if ITER:
        n_iters = ITER
    else:
        n_iters = 1
    for n in range(n_iters):
        mod = TinyVGG_1(input_shape=1,hidden_channels=HIDDEN_CHANNELS,output_shape=10).to(device)
        model_0.append(mod)
        optimizer_0.append(torch.optim.SGD(params=mod.parameters(),lr=LR))

    loss_fn = nn.CrossEntropyLoss()

    model_name = model_0[0]._get_name()
    #model_name = "Model_3"

    extra = ""
    namelist = [str(key)+"_"+str(val)+"_" for key,val in combination.items()]
    for elem in namelist:
        extra += elem
    extra = extra[:-1]

    # Create a writer to save results to tensorboard
    writer = create_writer(experiment_name=experiment_name,
                           model_name=model_name,
                           extra=extra)
    # Train the model for a number of epochs and log the results to tensorboard
    results = train(model=model_0,
                    train_dataloader=train_dataloader,
                    test_dataloader=test_dataloader,
                    optimizer=optimizer_0,
                    loss_fn=loss_fn,
                    epochs=NUM_EPOCHS,
                    iters=ITER,
                    writer=writer)
    # Store additional information on the experiment
    create_and_save_experiment_metadata(experiment_name=experiment_name,
                                        model_name=model_name,
                                        extra=extra,
                                        train_dataloader=train_dataloader,
                                        test_dataloader=test_dataloader,
                                        model=model_0[0],
                                        optimizer=optimizer_0[0],
                                        loss_fn=loss_fn,
                                        epochs=NUM_EPOCHS,
                                        hyperparameters_combination=combination
                                        )
    # Store the results of the accuracy and loss as a dataframe
    df = create_dataframe(results=results,
                          hyperparameters_combination=combination)
    save_dataframe(df=df,
                   model_name=model_name,
                   experiment_name=experiment_name,
                   extra=extra)


[INFO] Created SummaryWriter, saving to: c:\Users\oriol\GitHub_Repos\CNN-classification\experiment_logs\runs\Big_sweep\TinyVGG_1\Hidden_Channels_20_Epochs_3_lr_0.1_Iter_2_Size_0.05...
Iteration 1


  0%|          | 0/3 [00:00<?, ?it/s]

Iteration 2


  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Created experiment metadata, saving to: c:\Users\oriol\GitHub_Repos\CNN-classification\experiment_logs\results\Big_sweep\TinyVGG_1\Hidden_Channels_20_Epochs_3_lr_0.1_Iter_2_Size_0.05...
[INFO] Saving the above results to: c:\Users\oriol\GitHub_Repos\CNN-classification\experiment_logs\results\Big_sweep\TinyVGG_1\Hidden_Channels_20_Epochs_3_lr_0.1_Iter_2_Size_0.05\Results.feather
[INFO] Created SummaryWriter, saving to: c:\Users\oriol\GitHub_Repos\CNN-classification\experiment_logs\runs\Big_sweep\TinyVGG_1\Hidden_Channels_20_Epochs_3_lr_0.01_Iter_2_Size_0.05...
Iteration 1


...

[INFO] Created experiment metadata, saving to: c:\Users\oriol\GitHub_Repos\CNN-classification\experiment_logs\results\Big_sweep\TinyVGG_1\Hidden_Channels_40_Epochs_4_lr_0.01_Iter_2_Size_0.05...
[INFO] Saving the above results to: c:\Users\oriol\GitHub_Repos\CNN-classification\experiment_logs\results\Big_sweep\TinyVGG_1\Hidden_Channels_40_Epochs_4_lr_0.01_Iter_2_Size_0.05\Results.feather
[INFO] Created SummaryWriter, saving to: c:\Users\oriol\GitHub_Repos\CNN-classification\experiment_logs\runs\Big_sweep\TinyVGG_1\Hidden_Channels_40_Epochs_4_lr_0.001_Iter_2_Size_0.05...
Iteration 1


  0%|          | 0/4 [00:00<?, ?it/s]

Iteration 2


  0%|          | 0/4 [00:00<?, ?it/s]

[INFO] Created experiment metadata, saving to: c:\Users\oriol\GitHub_Repos\CNN-classification\experiment_logs\results\Big_sweep\TinyVGG_1\Hidden_Channels_40_Epochs_4_lr_0.001_Iter_2_Size_0.05...
[INFO] Saving the above results to: c:\Users\oriol\GitHub_Repos\CNN-classification\experiment_logs\results\Big_sweep\TinyVGG_1\Hidden_Channels_40_Epochs_4_lr_0.001_Iter_2_Size_0.05\Results.feather


# METADATA

In [3]:
def print_metadata(md):
    for key,value in zip(md.keys(),md.values()):
        print(f"{key}: {value}")

In [ ]:
from utils.helpers import retrieve_metadata

md_t3m = retrieve_metadata(experiment_name="Test_3_Models",model_name="TinyVGG_1",extra="Hidden_Channels_30_Epochs_3_lr_0.1_Iter_2_Size_0.11")

In [9]:
print_metadata(md_t3m)

date: 16-09-2025 00:32
experiment name: Test_3_models
model name: TinyVGG_1
learning rate: 0.1
epochs: 3
loss function: CrossEntropyLoss
optimizer name: SGD
dataset: FashionMNIST
training dataset size: 6600
testing dataset size: 1100
batch size: 32
Hyperparams: {'Hidden_Channels': 30, 'Epochs': 3, 'lr': 0.1, 'Iter': 2, 'Size': 0.11}
optimizer params: {'lr': 0.1, 'momentum': 0, 'dampening': 0, 'weight_decay': 0, 'nesterov': False, 'maximize': False, 'foreach': None, 'differentiable': False, 'fused': None}
model params: {'layer_1': Sequential(
  (0): Conv2d(1, 30, kernel_size=(3, 3), stride=(1, 1))
  (1): ReLU()
  (2): Conv2d(30, 30, kernel_size=(3, 3), stride=(1, 1))
  (3): ReLU()
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
), 'layer_2': Sequential(
  (0): Conv2d(30, 30, kernel_size=(3, 3), stride=(1, 1))
  (1): ReLU()
  (2): Conv2d(30, 30, kernel_size=(3, 3), stride=(1, 1))
  (3): ReLU()
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation

In [123]:
from utils.helpers import retrieve_results
from scipy.stats import pearsonr
from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import permutation_importance
import pandas as pd

results = retrieve_results(experiment_name="Big_sweep")


[INFO] retrieving results from c:\Users\oriol\GitHub_Repos\CNN-classification\experiment_logs\results


In [ ]:
col_y = "train_loss"
col_X = results.columns.to_list()[0:results.columns.to_list().index("train_loss")] 
# train loss is to separate HP, has nothing to do with col_y, they just happen to be the same
# col_y could be train_acc, test_loss...
col_X.remove("Iter")    # Iter has no importance since it generates statistically independent repetitions of a model
#col_X.remove("Epochs") # Keep epochs since it gives an idea of how relevant different total number of epochs are
col_X.append("Epoch #") # Adding Epoch # to see how it compares to Epochs
col_X[:] = [col for col in col_X if len(results[col].unique())>1] #type:ignore
# removes columns that only have one possible value since they cannot have an effect on importance or correlation 


In [40]:
X = results[col_X]      #type:ignore
y = results[[col_y]]    #type:ignore


In [45]:
r,p = pearsonr(X,y,axis=0)

In [49]:
print(f"{col_X}\n{r}\n{p}")

['Hidden_Channels', 'Epochs', 'lr', 'Epoch #']
[-0.05293559 -0.07230701 -0.82156094 -0.32461589]
[5.56068170e-01 4.21045311e-01 4.76380539e-32 2.08543109e-04]


In [32]:
col_X

['Hidden_Channels', 'Epochs', 'lr', 'Size', 'Epoch #']